In [50]:
import pandas as pd

supermarket_data = pd.read_csv('supermarket.csv') 
supermarket_data.dropna(inplace = True) 
supermarket_data.head()

,'grocery misc','baby needs','bread and cake','baking needs',coupons,juice-sat-cord-ms,tea,biscuits,'canned fish-meat','canned fruit',...,'750ml red nz','750ml white imp','750ml red imp','sparkling nz','sparkling imp','brew kits/accesry','port and sherry',ctrled label wine','non host support',total
0,?,t,t,t,?,t,?,t,?,?,...,?,?,?,?,?,?,?,?,?,high
1,?,?,?,?,?,?,?,?,t,t,...,?,?,?,?,?,?,?,?,?,low
2,?,?,t,t,?,t,?,t,?,t,...,?,?,?,?,?,?,?,?,?,low
3,?,?,t,t,?,t,?,t,?,?,...,?,?,?,?,?,?,?,?,?,low
4,?,?,t,t,?,t,t,?,?,?,...,?,?,?,?,?,?,?,?,?,low


In [51]:
supermarket_data.drop(columns = 'total',inplace=True)
columns_list = list(supermarket_data)
for i in range(len(supermarket_data)):
    for j in range(len(supermarket_data.columns)):
        if supermarket_data.iloc[i,j] == 't':
            supermarket_data.iloc[i,j] = columns_list[j]

supermarket_data = supermarket_data.values.tolist()
for i in range(len(supermarket_data)):
    supermarket_data[i] = list(filter(lambda x : x != '?', supermarket_data[i]))

In [52]:
item_list = supermarket_data

In [53]:
import pyfpgrowth

patterns = pyfpgrowth.find_frequent_patterns(item_list, 0.15*len(item_list))
rules = pyfpgrowth.generate_association_rules(patterns, 0.8)

In [54]:
patterns=sorted(patterns.items(),key=lambda x:x[1],reverse=True)
FP_Growth_support = pd.DataFrame(patterns)
FP_Growth_support.columns = ['items','support']
for i in range(len(FP_Growth_support)):
    FP_Growth_support.iloc[i,-1] = FP_Growth_support.iloc[i,-1]/len(item_list)
FP_Growth_support

,items,support
0,"('bread and cake',)",0.719628
1,"(fruit,)",0.640078
2,"('bread and cake', milk-cream)",0.505188
3,"('bread and cake', fruit)",0.502378
4,"('bread and cake', vegetables)",0.496541
...,...,...
1347,"(biscuits, confectionary, margarine)",0.150022
1348,"('baking needs', confectionary, juice-sat-cord...",0.150022
1349,"(beef, fruit, margarine)",0.150022
1350,"('baking needs', biscuits, cheese, vegetables)",0.150022


In [55]:
rules=sorted(rules.items(),key=lambda x:x[-1][-1],reverse=True)
rules

[(("'baking needs'", "'frozen foods'", 'biscuits', 'fruit', 'vegetables'),
  (("'bread and cake'",), 0.89937106918239)),
 (('biscuits', 'fruit', 'margarine', 'vegetables'),
  (("'bread and cake'",), 0.8947368421052632)),
 (("'frozen foods'", 'biscuits', 'fruit', 'vegetables'),
  (("'bread and cake'",), 0.8940269749518305)),
 (("'frozen foods'", 'biscuits', 'fruit', 'milk-cream'),
  (("'bread and cake'",), 0.8926673751328374)),
 (("'baking needs'", 'fruit', 'margarine', 'milk-cream'),
  (("'bread and cake'",), 0.8923821039903265)),
 (("'frozen foods'", 'biscuits', 'fruit', 'margarine'),
  (("'bread and cake'",), 0.8901234567901235)),
 (("'frozen foods'", "'tissues-paper prd'", 'biscuits', 'vegetables'),
  (("'bread and cake'",), 0.889168765743073)),
 (("'baking needs'", 'biscuits', 'margarine', 'milk-cream'),
  (("'bread and cake'",), 0.889168765743073)),
 (("'frozen foods'", 'biscuits', 'margarine', 'vegetables'),
  (("'bread and cake'",), 0.8876543209876543)),
 (("'frozen foods'", 'bi

In [87]:
FP_Growth_association_rule = pd.DataFrame(rules)

for i in range(len(FP_Growth_association_rule)):
    FP_Growth_association_rule.loc[i,'confidence'] = FP_Growth_association_rule.iloc[i,1][-1]
    FP_Growth_association_rule.iloc[i,1] = FP_Growth_association_rule.iloc[i,1][:-1]
FP_Growth_association_rule.columns = ['antecedents','consequents','confidence']
FP_Growth_association_rule


,antecedents,consequents,confidence
0,"('baking needs', 'frozen foods', biscuits, fru...",['bread and cake'],0.899371
1,"(biscuits, fruit, margarine, vegetables)",['bread and cake'],0.894737
2,"('frozen foods', biscuits, fruit, vegetables)",['bread and cake'],0.894027
3,"('frozen foods', biscuits, fruit, milk-cream)",['bread and cake'],0.892667
4,"('baking needs', fruit, margarine, milk-cream)",['bread and cake'],0.892382
...,...,...,...
358,"('canned fish-meat',)",['bread and cake'],0.801275
359,"('bread and cake', confectionary, vegetables)",[fruit],0.801124
360,"('bread and cake', 'canned vegetables', biscuits)",['frozen foods'],0.801093
361,"('baking needs', 'bread and cake', sauces-grav...",[fruit],0.800643
